In [ ]:
# Data Management
import pandas as pd
import numpy as np
# from pandas_datareader.data import DataReader
# from ta import  add_all_ta_features


# Statistics
from statsmodels.tsa.stattools import adfuller

# Data Preproccessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Supervised Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import RandomForestRegressor

# Graphing
import matplotlib.pyplot as plt

## Data Ingestions


In [ ]:
df = pd.read_csv("SydneyHousePrices.csv")


print(f"Length of data: {len(df)}" )
df

In [ ]:
# Interpertate Data
df.info()

# Feature Engineering

##### handle Non-Numerical Data

- Label Encoding
    - assigning each worded item in a column into a number that the ml algo can comprehend

- OneHot Encoding
    - get all unique values then make a chart of 0's and 1's wherever it showed

In [ ]:
# unique suburbs
suburbs_test_unique = df["suburb"].unique()

print("Unique Suburbs Length: " , len(suburbs_test_unique))
print("Preform Label Encoding")

# Label Encoding
labelencoder = LabelEncoder()
encoded_suburbs = labelencoder.fit_transform(df["suburb"])
df["suburbs_encoded"] = encoded_suburbs

df

In [ ]:
# Property Type
suburbs_test_unique = df["propType"].unique()

print("Unique Property Types Length: " , len(suburbs_test_unique))
print("Preform OneHot Encoding")


# OneHot Encoding
onehot_encoded = pd.get_dummies(df["propType"], prefix="pt", drop_first=True)
df = df.join(onehot_encoded)
df